In [5]:
# For Masking Images

In [1]:
# Importing required libraires
import os
import json
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

# Naming default directories names
images_directory="images"
masked_directory="masked"
source_path="dataset"
target_path="masked-dataset"

# Creating a directory
def create_dir(path):
    os.makedirs(path, exist_ok=True)

# Processing the data and saving the image and mask
def process_data(image_path, json_path, save_dir):
    
    # Reading the JSON file
    with open(json_path, "r") as f:
        data = json.load(f)

    # Loop over the JSON data (dictionary)
    for key, value in tqdm(data.items()):
        # Getting the filename
        filename = value["filename"]

        # Extracting the name of the image, by removing its extension
        name = os.path.splitext(filename)[0]

        # Reading the image
        image = cv2.imread(os.path.join(image_path, filename))

        H, W, _ = image.shape

        # Extracting information about the annotated regions
        regions = value["regions"]

        # Creating a blank mask
        mask = np.zeros((H, W, 3), dtype=np.uint8)

        # Loop over regions and draw on the mask
        for region in regions:
            points_x = np.array(region["shape_attributes"]["all_points_x"])
            points_y = np.array(region["shape_attributes"]["all_points_y"])
            points = np.array([(x, y) for x, y in zip(points_x, points_y)], dtype=np.int32)
            region_type = region["region_attributes"]["type"]

            # Assigning color based on region type
            if region_type == "bins":
                color = (255, 0, 0)  # Blue color
            elif region_type == "buildings":
                color = (0, 0, 255)  # Red color
            elif region_type == "trees":
                color = (0, 255, 0)  # Green color    
            else:
                color = (0, 255, 255)  # Yellow color: Just to check if some-items misses type while annotating

            # Drawing filled polygon on the mask
            cv2.fillPoly(mask, [points], color=color)

        # Saving the image and mask
        cv2.imwrite(os.path.join(save_dir, images_directory, f"{name}.jpg"), image)
        cv2.imwrite(os.path.join(save_dir, masked_directory, f"{name}.png"), mask)


# Dataset path which contains original images and json form annotator
dataset_path = source_path
dataset = glob(os.path.join(dataset_path))

# Loop over the dataset
for data in dataset:
    # Path for the files
    image_path = glob(os.path.join(data, "original*"))[0]
    json_path = glob(os.path.join(data, "*.json"))[0]

    # Creating directories to save the data
    dir_name = os.path.split(data)[-1]
    save_dir = os.path.join(target_path)
    create_dir(os.path.join(save_dir, images_directory))
    create_dir(os.path.join(save_dir, masked_directory))

    # Process the data
    process_data(image_path, json_path, save_dir)


100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [01:05<00:00,  1.96it/s]
